In [1]:
!conda install -y gdown

Retrieving notices: ...working... done
Channels:
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
 - pytorch
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    filelock-3.15.4            |     pyhd8ed1ab_0          17 KB  conda-forge
    gdown-5.2.0                |     pyhd8ed1ab_0          21 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          39 KB

The following NEW packages will be INSTALLED:

  filelock           conda-forge/noarch::filelock-3.15.4-pyhd8ed1ab_0 
  gdown              conda-forge/noarch::gdown-5.2.0-pyhd8ed1ab_0 



gdown-5.2.0          | 21 KB     |                                       |   0% 
filelock-3.15.4      | 17 KB     |                

In [ ]:
# import gdown
# url = "https://drive.google.com/uc?id=1BT-8zitp3cv8hy9U6ulK7y4M_VQAO65y/view?usp=sharing"

# output = 'dataset.zip'
# gdown.download(url, output)

# Downloading the dataset archive
!gdown --id 1BT-8zitp3cv8hy9U6ulK7y4M_VQAO65y

!unzip dataset.zip

In [3]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

# import keras
from keras.models import load_model, Model
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, ZeroPadding2D, MaxPooling2D, AveragePooling2D, Dense, Flatten, RandomRotation
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from PIL import Image
import numpy as np
import cv2 as cv

from timeit import default_timer as timer


2024-07-13 09:45:29.730335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 09:45:29.730450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 09:45:29.840290: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
dataset_path = "./dataset"

# Labels

In [5]:
# The labels are hardcoded for test purpose only, not for production intends


labels_file = open(os.path.join(dataset_path, "labels.data"), "r")
LABELS = {name:idx for idx, name in enumerate(labels_file.read().split('\n'))}

labels_file.close()

REVERSED_LABELS = {_[0]:_[1] for _ in [(value, key) for key, value in LABELS.items()]}


# Dataset loading functions

In [6]:

face_classifier = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")

def save_bounding_box(img:Image.Image, path:str, shape:tuple):

    img_mat = cv.cvtColor(np.array(img), cv.COLOR_RGB2BGR)
    gray_img = cv.cvtColor(img_mat, cv.COLOR_BGR2GRAY)

    faces = face_classifier.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40))

    faces_to_return = []
    root_pos = path.rfind('.')
    index = 0
    for (x, y, w, h) in faces:

        to_save = cv.resize(img_mat[y:y+h, x:x+w], shape)

        cv.imwrite(
           path[:root_pos] + str(index) + path[root_pos:],
           to_save
        )
        index += 1

        faces_to_return.append(Image.fromarray(cv.cvtColor(to_save, cv.COLOR_BGR2RGB)))

    return faces_to_return




def load(dir:str, shape:tuple=(224,224)) -> tuple:

    # Loading dataset
    data = []
    labels = []

    dir_content = os.listdir(dir)

    for person_folder in dir_content: # For each person's folder

        for _ in os.listdir(os.path.join(dir, person_folder)): # For face image in a person's folder

            if '.' in _: # If _ is actually a file
                file = Image.open(os.path.join(dir, person_folder, _))

                # If the loaded image doesn't meet the shape standards (maybe not cropped yet) we do so,
                # save the cropped version before adding to the dataset
                if file.size != shape:
                    # faces is a list consisting of Image objects of all the faces extrated in the current file
                    faces = save_bounding_box(file, os.path.join(dir, _), shape)

                    # Adding the face(s)
                    for f in faces:

                        data.append(np.asarray(f))

                        # Adding the label
                        for key in LABELS.keys():
                            if key in _:
                                labels.append(LABELS[key])
                                break

                    # Moving the old parent image
                    os.system("mkdir " + os.path.join(dir, "old_images").replace('/', '\\'))
                    # print(('move "' + os.path.join(dir, _) + '" "' + os.path.join(dir, "old_images/")).replace('/', '\\') + '"')
                    os.system(('move "' + os.path.join(dir, _) + '" "' + os.path.join(dir, "old_images/")).replace('/', '\\') + '"')

                else:

                    # Adding the file
                    data.append(np.asarray(file))

                    # Adding the label
                    for key in LABELS.keys():
                        if key.lower() == person_folder.lower() or person_folder.lower() in key.lower():
                            labels.append(LABELS[key])
                            break


    return np.array(data), to_categorical(labels)


def load_data(path:str="./dataset") -> tuple:

    train_data_path = os.path.join(path, "train_data")
    test_data_path = os.path.join(path, "test_data")

    # Loading training dataset
    train_data = load(train_data_path)
    # Loading training dataset
    test_data = load(test_data_path)

    return train_data, test_data




class TimingCallback(Callback):

    def __init__(self, logs={}):
        self.logs = []

    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()

    def on_epoch_end(self, epoch, logs=None):
        self.logs.append(timer() - self.starttime)


# Model implementation, loading and saving functions

In [29]:

#Implementation of convolution block
def convolutional_block(X, f, filters, stage, block, s):

    F1, F2, F3 = filters
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    return X



#Implementation of Identity Block

def identity_block(X, f, filters, stage, block):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Skip Connection
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X




#Implementation of ResNet-50

#Implementation of ResNet-50
def ResNet50(input_shape=(224, 224, 3)):

    X_input = Input(input_shape)

    # Data augmentation, clockwise rotation
    X = RandomRotation(1)(X_input)

    X = ZeroPadding2D((3, 3))(X)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[16, 16, 64], stage=2, block='a', s=1)
    X = identity_block(X, 3, [16, 16, 64], stage=2, block='a')
    X = identity_block(X, 3, [16, 16, 64], stage=2, block='b')
    X = identity_block(X, 3, [16, 16, 64], stage=2, block='c')
    # X = identity_block(X, 3, [16, 16, 64], stage=2, block='d')


    X = convolutional_block(X, f=3, filters=[32, 32, 128], stage=3, block='b', s=2)
    X = identity_block(X, 3, [32, 32, 128], stage=3, block='a')
    X = identity_block(X, 3, [32, 32, 128], stage=3, block='b')
    # X = identity_block(X, 3, [32, 32, 128], stage=3, block='c')
    # X = identity_block(X, 3, [32, 32, 128], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=4, block='c', s=2)
    X = identity_block(X, 3, [64, 64, 256], stage=4, block='a')
    # X = identity_block(X, 3, [64, 64, 256], stage=4, block='b')
    # X = identity_block(X, 3, [64, 64, 256], stage=4, block='c')
    # X = identity_block(X, 3, [64, 64, 256], stage=4, block='d')
    # X = identity_block(X, 3, [64, 64, 256], stage=4, block='e')
    # X = identity_block(X, 3, [64, 64, 256], stage=4, block='f')

    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=5, block='d', s=2)
    # X = identity_block(X, 3, [128, 128, 512], stage=5, block='a')
    # X = identity_block(X, 3, [128, 128, 512], stage=5, block='c')

    # Replacing the GAP with Flatten and Dense layers
    # X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
#     X = Flatten()(X)
#     X = Dense(64, activation='relu', name='fc0',kernel_initializer=glorot_uniform(seed=0))(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model




def create_model() -> Model:

    base_model = ResNet50(input_shape=(224, 224, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(x)
    x = Dense(64, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(x)
    x = Dense(33, activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(x)

    model = Model(inputs=base_model.input, outputs=x)

#     for layer in base_model.layers:
#         layer.trainable = False

    return model


def save_model(model:Model):

    model.save("base_model_og.keras")

def load_weights():

    return load_model("base_model_og.keras")


# Training phase

## Loading the dataset

In [8]:
print('\nLoading the dataset ...')
(X_train, y_train), (X_test, y_test) = load_data(dataset_path)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


Loading the dataset ...
(2053, 224, 224, 3) (2053, 33)
(620, 224, 224, 3) (620, 33)


In [9]:
mkdir models

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:

print('\nLoading the model ...')
model = create_model()

print('\nCompiling the model ...')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
time = TimingCallback()
filepath = "models/resnet50-{epoch}-loss-{loss:.2f}-accuracy-{accuracy:.2f}-val_accuracy-{val_accuracy:.2f}.keras"
# checkpoint = ModelCheckpoint(filepath, monitor="accuracy", verbose=1, save_best_only=True, mode='max')
checkpoint1 = ModelCheckpoint(filepath, monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor="val_accuracy", patience=20)


callbacks_list = [checkpoint1, time, earlystop]

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, shuffle=True, callbacks=callbacks_list)

print(f"\nEnd of training.\nThe training lasted: {sum(time.logs)} s.")

print("\nSaving...")
save_model(model)

print('\nModel saved.')



## Saving the feature extraction part

In [ ]:

output = None

for layer in model.layers:
    if isinstance(layer, Flatten):
        output = layer.output


if output:
    new_model = Model(inputs=model.input, outputs=output)

    new_model.save("FExtractor.keras")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(new_model.summary())
